In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
import os
import time
from sklearn.utils.class_weight import compute_class_weight
import shap
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from explainerdashboard import InlineExplainer, ExplainerDashboard, ClassifierExplainer
import random

In [4]:
class CorrEncoder:
    """
    CorrEncoder: Takes a dataset as input and uses it for the encode function. Encodes the filtered categories then draws correlations.
    If correlation is above the threshold adds it to a new dataframe then returns the one hot encoded values with the labels.

    Initialisation:
        - data (pd.DataFrame): The Dataset that contains the target column and target label variables.
    """
    
    def __init__(self, data):
        self.data = data.reset_index(drop=True).copy()
        #self.data = self.data.drop(columns=['Label'])

    def encode(self, target_column, sparse_n, threshold):
        """
        encode: Takes a target column and target label to encode and draw correlations from. The target column is iterated through
        for all categories that contain more positive values than defined in sparse_n. This allows for filtering of sparse categories.
        The function then one hot encodes the given category with the static target column and draws correlations for them. If correlation
        is greater then threshold then add it to the new DataFrame. The function returns the one hot encoded categories that pass the
        threshold with the target label.

        The purpose of this function is to resolve the high cardinality problem in one hot encoding.

        Parameters:
            - target_column (string): The name of the target column. The target column should contain the various categories to encode.
            - sparse_n (integer): The minimum amount of positive values required for a category after encoding (deals with sparse categories).
            - threshold (float): The threshold for correlation. The function creates onehot encoded columns of all variables that high correlation
              higher that the threshold to the target label.
            - cat (string): The category label to compare to.

        Returns:
            - ohe_df (pd.DataFrame): The one hot encoded values from the target columns.
        """


        self.data[target_column] = self.data[target_column].astype(str)
        value_counts = self.data[target_column].value_counts()
        # Check if number of 1s is above the given threshold set by sparse_n.
        categories = value_counts[value_counts > sparse_n].index.tolist()
        ohe_list = []    
        attack_cat = self.data['attack_cat']
        # Go through each unique category in the target column.
        for c in categories:
            col_name = f'{target_column}_{c}'

            # Create the binary encoding column for the current category and target label
            corr_column = (self.data[target_column] == c).astype(int)
            correlation = corr_column.corr(attack_cat)

            # Check if absolute correlation is greater than threshold.
            if abs(correlation) > threshold:
                corr_column.name = col_name
                ohe_list.append(corr_column)
        print('Number of Encoded Features for', target_column)
        print(len(ohe_list))
        if ohe_list:
            # NOTE: This section can be expanded to include print outs but at the moment am focusing on the evaluations.
            ohe_df = pd.concat(ohe_list, axis=1)
            return ohe_df
        else:
            # This ommits errors (if really high thresholds are used).
            print("No correlations exceed the threshold.")
            return pd.DataFrame()

In [5]:
data = pd.read_csv('../Cleaned_full_data.csv')
data = data.reset_index(drop=True)
# Set NA to 0.
data['ct_ftp_cmd'] = data['ct_ftp_cmd'].fillna(0)
data['ct_ftp_cmd'] = data['ct_ftp_cmd'].fillna(0)
data['attack_cat'] = data['attack_cat'].str.replace(r'\s+', '', regex=True)
data['attack_cat'] = data['attack_cat'].str.replace('Backdoors', 'Backdoor')
# Select a threat category.
category = 'DoS'
data['attack_cat'] = (data['attack_cat'] == category).astype(int)

C:\Users\adi_s\AppData\Local\Temp\ipykernel_16736\112763362.py:1: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.


In [7]:
encoder = CorrEncoder(data)
threshold = 0.01
ohe4a = encoder.encode('proto', 30, threshold)

Number of Encoded Features for proto
129


In [ ]:
# Although Dsport has just as many unique categories it is not as sensitive to threshold as sport.
encoded_columns = ['dsport', 'proto', 'sport', 'srcip', 'dstip', ]
for column in encoded_columns:
    unique_categories = data[column].nunique()
    print(f"Column '{column}': {unique_categories}")

Column 'dsport': 128310
Column 'proto': 135
Column 'sport': 100341
Column 'srcip': 43
Column 'dstip': 47


In [5]:
# Takes around 90 minutes (likely because many values are just above sparse_n in sport
# column considering the attack type).
encoder = CorrEncoder(data)
threshold = 0.01
# 0.001 fails with memory allocation for sport (concatenating the columns).
# Could try 0.005.
ohe1a = encoder.encode('sport', 30, threshold)
threshold = 0
ohe2 = encoder.encode('state', 30, threshold)
ohe3 = encoder.encode('service', 30, threshold)
ohe4 = encoder.encode('proto', 30, threshold)
# This could be reduced to 0 too but just want to see if we get an improvement with the current
# settings.
threshold = 0.001
ohe5 = encoder.encode('dsport', 30, threshold)
ohe6 = encoder.encode('srcip', 30, threshold)
ohe7 = encoder.encode('dstip', 30, threshold)

Number of Encoded Features for sport
11
Number of Encoded Features for state
9
Number of Encoded Features for service
13
Number of Encoded Features for proto
132
Number of Encoded Features for dsport
76
Number of Encoded Features for srcip
26
Number of Encoded Features for dstip
29


In [8]:
encoded_columns = {
    'proto2': ohe4a,
}

# Save data. This is the most optimal set I have been able to get without memory issues.
# NOTE: Batching may not be possible because we won't be drawing correlation on the full variable.
for column_name, encoded_data in encoded_columns.items():
    encoded_data.to_csv(f'DoS_{column_name}_encoded.csv', index=False)
    print(f"DoS_{column_name}_encoded.csv")

DoS_proto2_encoded.csv


In [12]:
data_encoded = pd.concat([data, ohe1a, ohe2, ohe3, ohe4, ohe5, ohe6, ohe7], axis=1)
data_encoded = data_encoded.drop(columns=['sport', 'state', 'service', 'proto', 'dsport', 'srcip', 'dstip'])

In [ ]:
print(len(data_encoded.columns))

338
